In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from utils_pcmci import *

import itertools
import pickle

In [2]:
#### Parameters to run ####
# seasons_mask = {'DJF': [12, 1, 2], 'MAM': [3, 4, 5], 'JJA': [6, 7, 8], 'SON': [9, 10, 11]}     
seasons_mask = {'FMA': [2, 3, 4], 'MJJ': [5, 6, 7], 'ASO': [8, 9, 10], 'NDJ': [11, 12, 1]} 
              
variables = [
    'sst', 
    # 'mtpr', 
    'prate',
    # 'msl'
    ]

model_name = {
    'sst': 'FULL_ERA5_SST_1940-2024_converted_detrend.nc',
    # 'mtpr': 'ERA5_mean_precipitation_1940-2024_converted_detrend.nc',
    'prate': 'PRATE_NCEP_NCAR_Reanalysis_1948-2024.nc',
    # 'msl': 'ERA5_mean_SLP_1940-2024_converted_detrend.nc'
    }

target = 'prate_FMA'
ip = 0
n_comps = {'sst': 15, 'prate': 5}
n_splits = 5
mask = 'unmasked'
method_arg = 'pcmci'
sequence_length = 12
period_length = 38 * 2
model = "_".join(variables)
months = "_".join(seasons_mask.keys())


# Define the number of components and the names of the components
total_comps = int((sum(n_comps.values()) / len(n_comps)) * len(seasons_mask) * len(model_name))

selected_components=['c'+str(i) for i in range(1,total_comps+1)] #  + 2 to include the component that is the precipitation rate
comp_names = [f'{var}_{months}' for months in seasons_mask.keys() for var in variables for _ in range(n_comps[var])]
comps_order_file = pd.DataFrame({'comp_number': ['c'+str(i) for i in range(1, total_comps+1)], # + 2 to include the component that is the precipitation rate
                                 'comps': [i for i in range(total_comps)], # + 1 to include the component that is the precipitation rate
                                 'name': [ i + '_' + j for i, j in zip(selected_components, comp_names)]})

In [3]:
# Load the 
datadict, fulldata, fulldata_mask = load_varimax_data(variables, seasons_mask, model_name, n_comps, mask)
print("Fulldata shape = %s" % str(fulldata.shape))
print("Fulldata masked shape = %s" % str(fulldata_mask.shape))

Fulldata shape = (912, 80)
Fulldata masked shape = (912, 80)


In [4]:
# Create dataframe with lagged variables
fulldata_pd = pd.DataFrame(fulldata, columns=comps_order_file['comp_number'])

def create_lagged_features(data, lags):
    temp = pd.DataFrame(index=data.index)
    for col in data.columns:
        for lag in range(1, lags+1):
            temp[f'{col}_lag_{lag}'] = data[col].shift(lag)
    
    return temp

lagged_df = create_lagged_features(fulldata_pd, 5)
combined_df = pd.concat([fulldata_pd, lagged_df], axis=1).dropna().reset_index(drop=True)

/var/folders/5s/g78x_2cn3b56vfckpn9d5vpr0000gn/T/ipykernel_47929/448644342.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp[f'{col}_lag_{lag}'] = data[col].shift(lag)
/var/folders/5s/g78x_2cn3b56vfckpn9d5vpr0000gn/T/ipykernel_47929/448644342.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  temp[f'{col}_lag_{lag}'] = data[col].shift(lag)
/var/folders/5s/g78x_2cn3b56vfckpn9d5vpr0000gn/T/ipykernel_47929/448644342.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inser

In [5]:
np.random.seed(123)

selected_components_lasso = {comp: {} for comp in comps_order_file['comps']}

for comp in selected_components_lasso.keys():
    print(f"Component {comp}")

    X = combined_df.drop(columns=[f'c{comp+1}'])
    y = combined_df[f'c{comp+1}']

    # Standardize the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Model with CV and fit
    tscv = TimeSeriesSplit(n_splits=5)
    lasso_cv = LassoCV(cv=tscv, random_state=123)
    lasso_cv.fit(X_scaled, y)

    # Selected alpha 
    selected_components_lasso[comp]['alpha'] = lasso_cv.alpha_
    print("Selected alpha:", round(lasso_cv.alpha_, 5))

    # Selected features and filter dataset to check on test set 
    selected_features = np.where(lasso_cv.coef_ != 0)[0]
    selected_components_lasso[comp]['selected_features'] = selected_features
    print("Selected features:", selected_features)

    split_idx = list(tscv.split(X_scaled))[-1][1][0]
    X_train_selected = X_scaled[:split_idx, selected_features]
    X_test_selected = X_scaled[split_idx:, selected_features]

    y_train, y_test = y[:split_idx], y[split_idx:]

    # Evaluate the model on the test set with selected features
    lasso_cv_selected = LassoCV(cv=tscv, random_state=12)
    lasso_cv_selected.fit(X_train_selected, y_train)
    y_pred_selected = lasso_cv_selected.predict(X_test_selected)
    
    r2_score_cv = lasso_cv_selected.score(X_test_selected, y_test)

    selected_components_lasso[comp]['r2_score'] = r2_score_cv
    print("Test set R^2 with selected features:", round(r2_score_cv, 5))
    print()

Component 0


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.176e+03, tolerance: 1.827e+03
  model = cd_fast.enet_coordinate_descent(


Selected alpha: 0.13279
Selected features: [  0   1   2   3   7   9  11  12  13  19  20  21  23  24  25  26  28  29
  30  32  34  38  41  43  44  46  52  53  57  59  62  63  64  67  68  69
  70  72  73  79  84  93  98  99 101 104 120 122 124 125 126 129 130 142
 148 149 151 165 177 188 198 201 203 204 206 207 208 228 231 234 236 239
 240 241 243 246 252 260 261 267 268 274 293 294 303 304 322 323 326 329
 330 339 341 346 347 353 354 360 366 368 370 371 372 373 398 408 412 422
 423 425 427 434 450 452 456 457 460 461 468 472]
Test set R^2 with selected features: 0.99687

Component 1


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.794e+04, tolerance: 1.534e+03
  model = cd_fast.enet_coordinate_descent(


Selected alpha: 0.12279
Selected features: [  0   2   3   8   9  10  12  13  14  21  22  23  26  28  30  32  33  38
  40  41  44  45  46  48  49  50  51  52  58  59  60  61  62  63  64  65
  66  67  68  69  72  73  77  84  88  93  96 102 103 114 119 126 135 136
 139 140 142 144 145 149 152 154 157 162 165 166 167 171 174 178 183 189
 190 196 197 206 208 209 223 229 231 232 233 234 235 237 238 244 253 255
 279 289 291 294 299 301 303 304 313 314 318 319 320 324 325 328 329 331
 332 333 339 345 346 349 362 365 366 370 373 374 375 376 386 400 402 408
 413 415 420 428 436 446 450 459 464 472]


...

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.14870434547788136, tolerance: 0.1457931008403157
  model = cd_fast.enet_coordinate_descent_gram(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.14930722555354592, tolerance: 0.1457931008403157
  model = cd_fast.enet_coordinate_descent_gram(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.19185079756683088, tolerance: 0.1457931008403157
  model = cd_fast.enet

Test set R^2 with selected features: 0.9949

Component 79


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0846637850561116, tolerance: 0.9245412906188526
  model = cd_fast.enet_coordinate_descent_gram(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.401e+00, tolerance: 1.159e+00
  model = cd_fast.enet_coordinate_descent(


Selected alpha: 0.00325
Selected features: [  6   8  15  16  17  18  30  31  32  37  38  39  48  50  52  53  55  57
  58  59  75  77  78  98  99 100 101 121 130 145 165 170 171 178 199 200
 201 225 227 252 268 278 314 315 328 336 339 342 343 358 360 364 365 370
 371 393 401 409 413 436 437 439 440 441 443 452 457 459 464]
Test set R^2 with selected features: 0.99761



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.929e+00, tolerance: 9.245e-01
  model = cd_fast.enet_coordinate_descent(


In [6]:
selected_components = ['c'+str(i) for i in range(1,total_comps+1)]

selected_components = []
for lag in range(0, 6):
    for i in range(total_comps):
        if lag == 0:
            selected_components.append(f'c{i+1}')
        else:  
            selected_components.append(f'c{i+1}_lag_{lag}')

for i in selected_components_lasso.keys():
    selected_components_lasso[i]['selected_features_names'] = [selected_components[i] for i in selected_components_lasso[i]['selected_features']]
    selected_components_lasso[i]['n_features'] = len(selected_components_lasso[i]['selected_features'])

selected_components_lasso

{0: {'alpha': 0.1327913233321651,
  'selected_features': array([  0,   1,   2,   3,   7,   9,  11,  12,  13,  19,  20,  21,  23,
          24,  25,  26,  28,  29,  30,  32,  34,  38,  41,  43,  44,  46,
          52,  53,  57,  59,  62,  63,  64,  67,  68,  69,  70,  72,  73,
          79,  84,  93,  98,  99, 101, 104, 120, 122, 124, 125, 126, 129,
         130, 142, 148, 149, 151, 165, 177, 188, 198, 201, 203, 204, 206,
         207, 208, 228, 231, 234, 236, 239, 240, 241, 243, 246, 252, 260,
         261, 267, 268, 274, 293, 294, 303, 304, 322, 323, 326, 329, 330,
         339, 341, 346, 347, 353, 354, 360, 366, 368, 370, 371, 372, 373,
         398, 408, 412, 422, 423, 425, 427, 434, 450, 452, 456, 457, 460,
         461, 468, 472]),
  'r2_score': 0.9968656527522426,
  'selected_features_names': ['c1',
   'c2',
   'c3',
   'c4',
   'c8',
   'c10',
   'c12',
   'c13',
   'c14',
   'c20',
   'c21',
   'c22',
   'c24',
   'c25',
   'c26',
   'c27',
   'c29',
   'c30',
   'c31',
   'c33

In [7]:
import pickle

file_name = './runs/pcmci_results/variable_selection_LASSO_%s_components-months-%s_%s.bin' % (total_comps, months, mask)
file_name = './runs/pcmci_results/variable_selection_LASSO_%s_3dm_%s-comps-%s_months-%s_%s.bin' % (model, total_comps, str(list(n_comps.values())), months, mask)

file = open(file_name, 'wb')
pickle.dump(selected_components_lasso, file, protocol=-1)        
file.close()